In [ ]:
import requests
import time
from datetime import datetime

class BlockBeatsMonitor:
    def __init__(self, interval=30):
        """
        :param interval: Checking interval in seconds. Recommended >= 10s to avoid IP blocking.
        """
        self.api_url = "https://api.theblockbeats.news/v1/open-api/open-flash"
        self.interval = interval
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Referer": "https://www.theblockbeats.info/"
        }
        # Set to store IDs of news that have already been processed to avoid duplicates
        self.seen_ids = set()
        self.is_first_run = True

    def fetch_latest_news(self):
        """Fetches the latest newsflash list from the API"""
        params = {"size": 10, "page": 1, "type": "all"}
        try:
            response = requests.get(self.api_url, params=params, headers=self.headers, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data.get("status") == 0:
                    return data.get("data", {}).get("data", [])
            return []
        except Exception as e:
            print(f"[{datetime.now().strftime('%H:%M:%S')}] Request failed: {e}")
            return []

    def start(self):
        """Starts the real-time monitoring loop"""
        print(f"🚀 BlockBeats Live Monitor Started...")
        print(f"⏰ Frequency: Once every {self.interval} seconds")
        print("-" * 60)

        while True:
            news_list = self.fetch_latest_news()
            
            if news_list:
                # Process items in reverse (oldest to newest)
                new_items = []
                for item in reversed(news_list):
                    news_id = item.get("id")
                    if news_id not in self.seen_ids:
                        new_items.append(item)
                        self.seen_ids.add(news_id)

                # On first run, just initialize seen_ids without pushing notifications
                if self.is_first_run:
                    latest_id = news_list[0].get('id') if news_list else 'None'
                    print(f"Initialized. Latest news ID recorded: {latest_id}")
                    self.is_first_run = False
                else:
                    for news in new_items:
                        self.push_notification(news)

            # Keep seen_ids size manageable (maintain last 100 IDs)
            if len(self.seen_ids) > 100:
                # Keep the 50 most recently added items
                current_ids_list = list(self.seen_ids)
                self.seen_ids = set(current_ids_list[-50:])

            time.sleep(self.interval)

    def push_notification(self, news):
        """
        Handles new news alerts. 
        You can integrate Telegram, Discord, or Slack webhooks here.
        """
        title = news.get("title", "No Title")
        content = news.get("content", "").replace("\n", " ")
        timestamp = int(news.get("create_time", 0))
        pub_time = time.strftime("%H:%M:%S", time.localtime(timestamp))
        
        # Build direct link to the flash news
        news_id = news.get('id')
        link = news.get("url", f"https://www.theblockbeats.info/flash/{news_id}")

        print(f"\n🔔 [NEW FLASH] {pub_time}")
        print(f"Title: {title}")
        print(f"Summary: {content[:150]}...")
        print(f"Link: {link}")
        print("-" * 40)

        # Example: Implement your webhook push here
        # self.send_to_webhook(title, content, link)

if __name__ == "__main__":
    # Initialize monitor with a 30-second interval
    monitor = BlockBeatsMonitor(interval=30)
    monitor.start()

🚀 BlockBeats Live Monitor Started...
⏰ Frequency: Once every 30 seconds
------------------------------------------------------------
Initialized. Latest news ID recorded: 330137
